# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil
import joblib

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'Hyperdrive-exp'

experiment=Experiment(ws, experiment_name)

quick-starts-ws-142076
aml-quickstarts-142076
southcentralus
6971f5ac-8af1-446e-8034-05acea24681f


## Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset is loaded in the supporting train.py script using a URL.

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

Logisitic Regression was chosen because it is a widely used classifier suitable for binary classification problems.

The model is trained using different combinations of C and max_iter hyperparameters. C is inverse of regularization strength. Smaller values specify stronger regularization. max_iter is the maximum number of iterations taken for the solvers to converge.

The best C and max_iter combination to have a correct hyperparameter model which is trained with the right number of iterations it requires to converge and does not suffer from overfitting to show a good generalisation performance on the test data is found.

BanditPolicy was chosen as the early termination policy, since it is based on slack criteria, and a frequency and delay interval for evaluation. Early termination is required for RandomParameterSampling. It helps in saving time and computational power by discarding poorly performing models early on.

Additionally, in configuration settings, weighted under the curve AUC_weighted was specified as the primary metric to optimise because it handles unbalanced data. The sampling method was defined over the search space: RandomParameterSampling. This method chooses randomly the best hyperparameters to optimise for speed.

In [4]:
# An early termination policy. 
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


# Different parameters that will be used during training
param_sampling = RandomParameterSampling( 
    {
    '--C': choice(0.0001, 0.001, 0.01, 0.1, 1,10,100,1000),
    '--max_iter': choice(100, 200, 300, 400, 500)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
shutil.copy('./train.py', script_folder)


# Estimator and hyperdrive config
estimator = SKLearn(source_directory = script_folder, 
              compute_target=cpu_cluster, 
              entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, 
                          hyperparameter_sampling=param_sampling,
                          policy=early_termination_policy, 
                          primary_metric_name = "AUC_weighted", 
                          primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs = 15)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config = hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_dcadb89d-284d-4433-97ef-f57cb9210320
Web View: https://ml.azure.com/experiments/Hyperdrive-exp/runs/HD_dcadb89d-284d-4433-97ef-f57cb9210320?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-142076/workspaces/quick-starts-ws-142076

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-06T10:56:01.875925][GENERATOR][INFO]Trying to sample '15' jobs from the hyperparameter space<END>\n""<START>[2021-04-06T10:56:01.363472][API][INFO]Experiment created<END>\n""<START>[2021-04-06T10:56:02.136088][GENERATOR][INFO]Successfully sampled '15' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-06T10:56:03.7057292Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_dcadb89d-284d-4433-97ef-f57cb9210320
Web View: https://ml.azure.com/experiments/Hyperdrive-exp/runs/HD_dcadb89d-284d-4433-97ef-f57cb9210320?wsid=/subscriptions/6971f

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
# Getting the best run and displaying its properties
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)
print('Best Run Id: ', best_run.id)
print('AUC_weighted:', best_run_metrics['AUC_weighted'])

['--C', '0.1', '--max_iter', '100']
Best Run Id:  HD_dcadb89d-284d-4433-97ef-f57cb9210320_6
AUC_weighted: 0.8727777777777778


In [8]:
#TODO: Save the best model
best_run.download_file("/outputs/model.joblib", "Logistic_Reg_Heart_Failure.joblib")

#Register the model
os.makedirs('outputs', exist_ok=True)
model_hd = best_run.register_model(model_name = 'HD_best_model', model_path = './outputs/model_hd.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service